# "US Presidential Candidate Tweets"
> "A visualization of recent tweets from candidates in the 2020 US Presidential Election"

- toc: false
- branch: master
- badges: true
- comments: true
- categories: [Data Visualization]
- hide: false
- search_exclude: true

In [65]:
#collapse
from textblob import TextBlob
import altair as alt
import pandas as pd
import datetime

def generate_date_list(num_days):
    base = datetime.date.today()
    date_list = [(base - datetime.timedelta(days=x)).strftime('%Y-%m-%d') for x in range(num_days)]
    return date_list

def add_columns_to_df(df):  
    df['created_at'] = pd.to_datetime(df['created_at'])
    df['username_label'] = '@' + df['username']
    df['created_at_est'] = df['created_at'].dt.tz_localize('UTC').dt.tz_convert('US/Eastern')
    df['sentiment'] = df['tweet_text'].apply(lambda tweet: TextBlob(tweet).sentiment.polarity)
    df['subjectivity'] = df['tweet_text'].apply(lambda tweet: TextBlob(tweet).sentiment.subjectivity)
    df = df.assign(color=['steelblue' if user == 'JoeBiden' else 'red' for user in df['username']])
    return df

def generate_df(usernames, number_of_days):
    url = "https://raw.githubusercontent.com/joshkraft/daily-candidate-tweets/main/data/"
    usernames = ["realDonaldTrump", "JoeBiden"]
    dates = generate_date_list(number_of_days)
    df = pd.DataFrame()

    for username in usernames:
        for date in dates:
            file_location = url + username + "/" + str(date) + ".csv"
            try:
                data = pd.read_csv(file_location)
                df = df.append(data)
            except:
                pass

    return df

df = generate_df(["realDonaldTrump", "JoeBiden"], 7)
df = add_columns_to_df(df)

In [70]:
#collapse
# THEME SOURCE: https://github.com/chekos/altair_themes_blog/blob/master/notebooks/vox_theme.py
def vox_theme():
    markColor = '#3e5c69'
    
    return {
        'config': {
            'background': '#fff',
            'arc': { 
                'fill': markColor 
            },
            'area': { 
                'fill': markColor 
            },
            'line': { 
                'stroke': markColor 
            },
            'path': { 
                'stroke': markColor 
            },
            'rect': { 
                'fill': markColor 
            },
            'shape': { 
                'stroke': markColor 
            },
            'symbol': { 
                'fill': markColor 
            },
            'axis': {
                'domainWidth': 0.5,
                'grid': True,
                'labelPadding': 2,
                'tickSize': 5,
                'tickWidth': 0.5,
                'titleFontWeight': 'normal',
            },
            'axisBand': {
                'grid': False,
            },
            'axisX': {
                'gridWidth': 0.2,
            },  
            'axisY': {
                'gridDash': [3],
                'gridWidth': 0.4,
            },
            'legend': {
                'labelFontSize': 14,
                'padding': 1,
                'symbolType': 'square',
            },
            'range': {
                'category': [
                    '#3e5c69',
                    '#6793a6',
                    '#182429',
                    '#0570b0',
                    '#3690c0',
                    '#74a9cf',
                    '#a6bddb',
                    '#e2ddf2',
                ],
            },
        }
        
    }


alt.themes.register('vox_theme', vox_theme)

alt.themes.enable('vox_theme')

ThemeRegistry.enable('vox_theme')

In [120]:
#collapse

click = alt.selection_multi(encodings=['color'])

brush = alt.selection(type='interval', encodings=['x'])

    
base = alt.Chart(df).mark_circle(opacity=0.5, size=120).encode(
            x = alt.X('created_at_est:T',
                axis = alt.Axis(title='Drag to Select Interval')),
            y = alt.Y('sentiment:Q',
                axis = alt.Axis(title='', tickCount=0, grid=False), 
                scale = alt.Scale(domain=(-1.1, 1.1))),
            color = alt.Color('color', scale=None),
        ).transform_filter(
            click
        ).properties(
            width=800,
            height=500
        )

upper = base.encode(
            x = alt.X('created_at_est:T',
                axis = alt.Axis(title='Tweeted At', labels=True),
                scale=alt.Scale(domain=brush)),
            y = alt.Y('sentiment:Q',
                axis = alt.Axis(title='Sentiment (Negative -> Positive)',
                                grid=False), 
                scale = alt.Scale(domain=(-1, 1))),
            tooltip = ['username','tweet_text'],
        ).add_selection(alt.selection_single()
        ).properties(
            title='Recent Tweets from @JoeBiden + @realDonaldTrump'
        )


threshholds = pd.DataFrame([{'positive_min': .5,
                             'positive_max': .7,
                             'negative_min': -.5,
                             'negative_max': -.7}])
positive_line = alt.Chart(threshholds).mark_rect(color='#488f31', opacity=0.1).encode(
    y='positive_min:Q',
    y2='positive_max:Q',
)

negative_line = alt.Chart(threshholds).mark_rect(color='#f59b56', opacity=0.1).encode(
    y='negative_min:Q',
    y2='negative_max:Q'
)

lower = base.properties(
            height=40
        ).add_selection(brush)
    
legend = alt.Chart(df).mark_rect().encode(
    y=alt.Y('username_label:N', axis=alt.Axis(title='')),
    color=alt.condition(click, 'color', alt.value('lightgray'), legend=None, scale=None),
    size=alt.value(150)
).properties(
    selection=click,
    title="Click to Select"
)


(upper | legend) & lower

alt.VConcatChart(...)